In [3]:
import xml.etree.ElementTree as ET 
import pandas as pd

In [4]:
def parse_kml(file_path):
    # 1. Đọc và parse file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # 2. Namespace của KML
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    records = []
    # 3. Duyệt từng Placemark
    for pm in root.findall('.//kml:Placemark', ns):
        rec = {}

        # — Lấy name & description
        name_el = pm.find('kml:name', ns)
        rec['name'] = name_el.text if name_el is not None else ''

        desc_el = pm.find('kml:description', ns)
        rec['description'] = desc_el.text if desc_el is not None else ''

        # — Lấy tọa độ (longitude, latitude, [alt])
        coord_el = pm.find('.//kml:coordinates', ns)
        if coord_el is not None and coord_el.text:
            parts = coord_el.text.strip().split(',')
            rec['longitude'] = float(parts[0])
            rec['latitude']  = float(parts[1])
        else:
            rec['longitude'] = rec['latitude'] = None

        # — Lấy các field trong <ExtendedData>
        #    Mỗi <Data name="..."><value>...</value>
        for data_el in pm.findall('.//kml:ExtendedData/kml:Data', ns):
            key = data_el.get('name')
            val_el = data_el.find('kml:value', ns)
            rec[key] = val_el.text if val_el is not None else ''

        records.append(rec)

    # 4. Chuyển list dict thành DataFrame
    df = pd.DataFrame(records)
    return df

if __name__ == '__main__':
    # Đường dẫn đến file KML
    kml_file = 'Trạm sạc VinFast.kml'
    df = parse_kml(kml_file)

    # In thử 5 dòng đầu
    print(df.head())

                         name  \
0        Vincom Plaza Hà Tĩnh   
1             Vinhomes Marina   
2  Vinpearl Waterpark Hà Tĩnh   
3  Vinpearl Discovery Cửa Hội   
4             Vincom+ Nam Đàn   

                                         description   longitude   latitude  \
0  Latitude: 18.3367<br>Longitude: 105.895203<br>...  105.895203  18.336700   
1  Latitude: 20.820971<br>Longitude: 106.689148<b...  106.689148  20.820971   
2  Latitude: 18.48855<br>Longitude: 105.891556<br...  105.891556  18.488550   
3  Latitude: 18.770851<br>Longitude: 105.75724<br...  105.757240  18.770851   
4  Latitude: 18.706059<br>Longitude: 105.494881<b...  105.494881  18.706059   

    Latitude   Longitude                                            Địa Chỉ  \
0    18.3367  105.895203  Vincom Plaza Hà Tĩnh, ngã tư đường Hàm Nghi gi...   
1  20.820971  106.689148  Vinhomes Marina, Đường Cầu Rào 2, phường Vĩnh ...   
2   18.48855  105.891556  Bãi đỗ xe ngoài trời Wertepak Hà Tĩnh, xóm hòa...   
3  18.7708

In [5]:
df.to_csv('Vinfast_station.csv',index=False)